In [ ]:
"""
Created on Mon Jul 17 14:42 2023

This script is to check the T and S profiles from CMIP

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob
import multimelt.melt_functions as meltf

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_simple_all = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_06161821/'


In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_isf+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)


In [ ]:
inputpath_profiles='/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'
T_ISMIP = xr.open_dataset(inputpath_profiles + 'T_mean_prof_50km_contshelf_ISMIP.nc')
S_ISMIP = xr.open_dataset(inputpath_profiles + 'S_mean_prof_50km_contshelf_ISMIP.nc')
TS_ISMIP = xr.merge([T_ISMIP.rename({'thetao':'theta_ocean'}),S_ISMIP.rename({'so':'salinity_ocean'})])

In [ ]:
scenario = 'historical'
yystart = 1980
yyend = 2014
ano_ISMIP = False

all_TS_list = []
for mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
           'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H','IPSL-CM6A-LR','MPI-ESM1-2-HR',
           'MRI-ESM2-0','UKESM1-0-LL']:
    print(mod)
    
    inputpath_profiles = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'

    T_Clim = xr.open_dataset(inputpath_profiles + 'T_mean_prof_50km_contshelf_'+mod+'_clim.nc')
    S_Clim = xr.open_dataset(inputpath_profiles + 'S_mean_prof_50km_contshelf_'+mod+'_clim.nc')
    TS_Clim = xr.merge([T_Clim.rename({'thetao':'theta_ocean'}),S_Clim.rename({'so':'salinity_ocean'})])

    file_TS_list = []
    for tt in tqdm(range(yystart,yyend+1)):
        file_T_orig = xr.open_dataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
        file_S_orig = xr.open_dataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
        file_TS_orig = xr.merge([file_T_orig.rename({'thetao':'theta_ocean'}), file_S_orig.rename({'so':'salinity_ocean'})]).sel(Nisf=rignot_isf).assign_coords({'time': tt})
        file_TS_list.append(file_TS_orig)

    file_TS_0 = xr.concat(file_TS_list, dim='time')
    if not ano_ISMIP:
        file_TS = (file_TS_0 - TS_ISMIP + TS_Clim).rename({'z':'depth'})
    else:
        file_TS = (file_TS_0 - TS_ISMIP*0 + TS_Clim*0).rename({'z':'depth'})

    file_TS['depth'] = -1*file_TS['depth']
    all_TS_list.append(file_TS.assign_coords({'model':mod}))
all_TS_notISMIP = xr.concat(all_TS_list, dim='model')

In [ ]:
scenario = 'historical'
yystart = 1980
yyend = 2014
ano_ISMIP = True

all_TS_list = []
for mod in ['ACCESS-CM2','ACCESS-ESM1-5','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
           'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H','IPSL-CM6A-LR','MPI-ESM1-2-HR',
           'MRI-ESM2-0','UKESM1-0-LL']:
    print(mod)
    
    inputpath_profiles = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'

    T_Clim = xr.open_dataset(inputpath_profiles + 'T_mean_prof_50km_contshelf_'+mod+'_clim.nc')
    S_Clim = xr.open_dataset(inputpath_profiles + 'S_mean_prof_50km_contshelf_'+mod+'_clim.nc')
    TS_Clim = xr.merge([T_Clim.rename({'thetao':'theta_ocean'}),S_Clim.rename({'so':'salinity_ocean'})])

    file_TS_list = []
    for tt in tqdm(range(yystart,yyend+1)):
        file_T_orig = xr.open_dataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
        file_S_orig = xr.open_dataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
        file_TS_orig = xr.merge([file_T_orig.rename({'thetao':'theta_ocean'}), file_S_orig.rename({'so':'salinity_ocean'})]).sel(Nisf=rignot_isf).assign_coords({'time': tt})
        file_TS_list.append(file_TS_orig)

    file_TS_0 = xr.concat(file_TS_list, dim='time')
    if not ano_ISMIP:
        file_TS = (file_TS_0 - TS_ISMIP + TS_Clim).rename({'z':'depth'})
    else:
        file_TS = (file_TS_0 - TS_ISMIP*0 + TS_Clim*0).rename({'z':'depth'})

    file_TS['depth'] = -1*file_TS['depth']
    all_TS_list.append(file_TS.assign_coords({'model':mod}))
all_TS_anoISMIP = xr.concat(all_TS_list, dim='model')

In [ ]:
file_TS_cut_depth_notISMIP = file_TS.where(file_TS_notISMIP.z >= -1*file_isf['front_bot_depth_max'])

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

reg_list = file_isf['region'].astype(str)

alpha_val = 0.07

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

for kisf in tqdm(sorted_isf_rignot):

    ax[i] = f.add_subplot(8,8,i+1)

    for tt in all_TS_anoISMIP.time:
        
        for mmod in all_TS_anoISMIP.model:
            
            T_data1 = all_TS_anoISMIP['theta_ocean'].sel(time = tt, Nisf = kisf, model = mmod)
            T_data2 = all_TS_notISMIP['theta_ocean'].sel(time = tt, Nisf = kisf, model = mmod)

            ax[i].plot(T_data1,-T_data1.depth, color='deepskyblue', alpha=alpha_val)
            ax[i].plot(T_data2,-T_data2.depth, color='orange', alpha=alpha_val)


            #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
            #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
            #leg_hdl.append(leg)

    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(-2,2)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'T_profiles_CMIP_'+scenario+'_all.png', dpi=300)

In [ ]:
TF_anoISMIP = all_TS_anoISMIP['theta_ocean'] - meltf.freezing_temperature(all_TS_anoISMIP['salinity_ocean'], -all_TS_anoISMIP['depth'])
TF_notISMIP = all_TS_notISMIP['theta_ocean'] - meltf.freezing_temperature(all_TS_notISMIP['salinity_ocean'], -all_TS_notISMIP['depth']) 

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

reg_list = file_isf['region'].astype(str)

alpha_val = 0.07

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

for kisf in tqdm(sorted_isf_rignot):

    ax[i] = f.add_subplot(8,8,i+1)

    for tt in all_TS_anoISMIP.time:
        
        for mmod in all_TS_anoISMIP.model.drop_sel(model='GISS-E2-1-H'):
            
            T_data1 = TF_anoISMIP.sel(time = tt, Nisf = kisf, model = mmod)
            T_data2 = TF_notISMIP.sel(time = tt, Nisf = kisf, model = mmod)

            ax[i].plot(T_data1,-T_data1.depth, color='deepskyblue', alpha=alpha_val)
            ax[i].plot(T_data2,-T_data2.depth, color='orange', alpha=alpha_val)


            #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
            #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
            #leg_hdl.append(leg)

    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(-2,2)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'thermalforcing_profiles_CMIP_'+scenario+'_all.png', dpi=300)

In [ ]:
kisf = 11
for mmod in all_TS_anoISMIP.model:
    plt.figure()
    for tt in all_TS_anoISMIP.time:
        T_data2 = TF_notISMIP.sel(time = tt, Nisf = kisf, model = mmod)
        plt.plot(T_data2,-T_data2.depth, color='orange', alpha=alpha_val)
    plt.title(mmod.values)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

reg_list = file_isf['region'].astype(str)

alpha_val = 0.07

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

for kisf in tqdm(file_isf.Nisf):

    ax[i] = f.add_subplot(8,8,i+1)

    for tt in file_TS.time:

        T_data = file_TS_cut_depth['so'].sel(time = tt, Nisf = kisf)
        ax[i].plot(T_data,T_data.z, color='grey', alpha=alpha_val)
        


        #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
        #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
        #leg_hdl.append(leg)

    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(33,35)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
#f.savefig(plot_path + 'S_profiles_CMIP_'+mod+'_'+scenario+'_all.png', dpi=300)